In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#!cp /content/gdrive/MyDrive/2390_Qi_Tian/2390--Spring2022/Week03/*.mat /content/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import welch, get_window
import scipy.io as scio
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
samplingFrequency = 500
sub1_s1_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series1_data.csv')
sub1_s1_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series1_events.csv')
sub1_s1_events = sub1_s1_events.to_numpy()[:,1:].astype(int)

sub1_s2_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series2_data.csv')
sub1_s2_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series2_events.csv')
sub1_s2_events = sub1_s2_events.to_numpy()[:,1:].astype(int)

sub1_s3_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series3_data.csv')
sub1_s3_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series3_events.csv')
sub1_s3_events = sub1_s3_events.to_numpy()[:,1:].astype(int)

sub1_s4_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series4_data.csv')
sub1_s4_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series4_events.csv')
sub1_s4_events = sub1_s4_events.to_numpy()[:,1:].astype(int)

sub1_s5_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series5_data.csv')
sub1_s5_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series5_events.csv')
sub1_s5_events = sub1_s5_events.to_numpy()[:,1:].astype(int)

sub1_s6_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series6_data.csv')
sub1_s6_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series6_events.csv')
sub1_s6_events = sub1_s6_events.to_numpy()[:,1:].astype(int)

sub1_s7_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series7_data.csv')
sub1_s7_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series7_events.csv')
sub1_s7_events = sub1_s7_events.to_numpy()[:,1:].astype(int)

sub1_s8_data = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series8_data.csv')
sub1_s8_events = pd.read_csv('/content/gdrive/MyDrive/2390_GroupProject/data/sub1/subj1_series8_events.csv')
sub1_s8_events = sub1_s8_events.to_numpy()[:,1:].astype(int)




In [ ]:
print(sub1_s1_events.shape[0]+sub1_s2_events.shape[0]+sub1_s3_events.shape[0]+sub1_s4_events.shape[0]+sub1_s5_events.shape[0]+sub1_s6_events.shape[0]+sub1_s7_events.shape[0]+sub1_s8_events.shape[0])


1422392


In [ ]:
labels = np.concatenate([sub1_s1_events,sub1_s2_events,sub1_s3_events,sub1_s4_events,sub1_s5_events,sub1_s6_events,sub1_s7_events,sub1_s8_events])
labels = labels[:1422000]
#labels = labels.reshape(-1,500,6)
#labels = labels[:,0,:]

#labels = labels[:,0]
print(labels.shape)

(1422000, 6)


In [ ]:
data = pd.concat([sub1_s1_data,sub1_s2_data,sub1_s3_data,sub1_s4_data,sub1_s5_data,sub1_s6_data,sub1_s7_data,sub1_s8_data])

data = np.transpose(data.to_numpy()[:,1:])
data = data.astype(float)

mean = np.mean(data,axis=1)
std = np.std(data,axis=1)
data = np.transpose(data)-mean


data = data/std
data = data[:1422000,:]
print(data)
print(data.shape)

#data = data.reshape(32,-1,500)
#print(data.shape)


[[-0.54409792  0.2223437  -0.77413519 ...  0.70542445  0.74976174
   2.70760351]
 [-0.53920664  0.15129877 -0.74927887 ...  0.52711447  0.56148569
   2.85887406]
 [-0.88893327 -0.06521913 -1.30108937 ...  0.52711447  0.45462631
   2.5838367 ]
 ...
 [-0.30931641 -0.81288249 -0.21238215 ... -0.49259569  0.3935638
  -1.07416022]
 [-0.45360922 -0.99218637 -0.37643393 ... -0.58175067  0.19002212
  -1.19792704]
 [-0.58322818 -0.80273321 -0.67470988 ... -0.40901289  0.2968815
  -1.05124044]]
(1422000, 32)


In [ ]:
print(data.shape)
data = data.reshape(-1,500,32)
print(data.shape)
print(data[:,:,0])

(1422000, 32)
(2844, 500, 32)
[[-0.54409792 -0.53920664 -0.88893327 ...  0.32655018  0.19448558
   0.21649635]
 [ 0.32410454  0.72029834  0.83279781 ...  0.93551472  0.94774293
   0.9819819 ]
 [ 0.95752549  0.81567833  0.95997113 ...  0.7961132   0.57600553
   0.62491835]
 ...
 [-0.84002045 -0.81800969 -0.78377072 ... -0.79844456 -0.73974918
  -0.50496767]
 [-0.45360922 -0.61013022 -0.70551021 ... -0.66393432 -0.52942408
  -0.55388049]
 [-0.44627229 -0.33621846 -0.38513128 ... -0.30931641 -0.45360922
  -0.58322818]]


In [ ]:
labels = labels[range(0,1422000,500),:]
print(labels.shape)

(2844, 6)


In [ ]:
def hjorth_params(x, axis=-1):
    x = np.asarray(x)
    dx = np.diff(x, axis=axis)
    ddx = np.diff(dx, axis=axis)
    x_var = np.var(x, axis=axis)  
    dx_var = np.var(dx, axis=axis)
    ddx_var = np.var(ddx, axis=axis)
    mob = np.sqrt(dx_var / x_var)
    com = np.sqrt(ddx_var / dx_var) / mob
    act = x_var
    return mob, com, act

print(data[0,:,0].shape)
mob, com, act = hjorth_params(data[0,:,0])
print(mob)
print(com)
print(act)

(500,)
0.4729360868639945
2.5498653484249143
0.13895993893578693


In [ ]:
fs = 500
window = get_window('hamming', fs)
nfft = fs
step = fs // nfft



segments = [[] for _ in range(2844)]

for j in range(32):
  for i in range(data.shape[0]):
    mob, com, act = hjorth_params(data[i,:,j])
    freq, psd = welch(data[i,:,j], window=window, fs=fs, nfft=nfft)
    total_energy = np.trapz(psd, freq)
    segments[i].append(np.trapz(psd[0:4 + step], range(0, 4 + step, step)) / total_energy)
    segments[i].append(np.trapz(psd[4:8 + step], range(4, 8 + step, step)) / total_energy)
    segments[i].append(np.trapz(psd[8:13 + step], range(8, 13 + step, step)) / total_energy)
    segments[i].append(np.trapz(psd[13:35 + step], range(13, 35 + step, step)) / total_energy)
    segments[i].append(mob)
    segments[i].append(com)
    segments[i].append(act)
    
features = np.asarray(segments).astype(float)

print(features.shape)

df1 = pd.DataFrame(features)
df2 = pd.DataFrame(labels,columns=['a','b','c','d','e','f'])

df = pd.concat([df1,df2],axis=1)



(2844, 224)


In [ ]:
print(features.shape)

(2844, 224)


In [ ]:
df.to_csv('seven_features',index=False)

In [ ]:
!wget -c http://h2o-release.s3.amazonaws.com/h2o/rel-zahradnik/1/h2o-3.30.0.1.zip
!unzip -n h2o-3.30.0.1.zip
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip



--2022-04-21 20:35:13--  http://h2o-release.s3.amazonaws.com/h2o/rel-zahradnik/1/h2o-3.30.0.1.zip
Resolving h2o-release.s3.amazonaws.com (h2o-release.s3.amazonaws.com)... 54.231.132.121
Connecting to h2o-release.s3.amazonaws.com (h2o-release.s3.amazonaws.com)|54.231.132.121|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  h2o-3.30.0.1.zip
--2022-04-21 20:35:14--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.19M  40.6MB/s    in 0.3s    

2022-04-21 20:35:14 (40.6 MB/s) - ‘ngrok-stable-linux-amd64

In [ ]:
get_ipython().system_raw('./ngrok authtoken 24seBzMtkgTY7nHCeghpzT7fNrP_4Tp56NPwWDh5oaT2eGCxt')  #This applies the ngrok authtoken


In [ ]:
get_ipython().system_raw('./ngrok http 54321 &')  #This exposes the h2o FLOW user interface to build models interactively 


In [ ]:
get_ipython().system_raw('java -jar h2o-3.30.0.1/h2o.jar &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://3180-35-239-235-30.ngrok.io


In [ ]:
!pip install https://h2o-release.s3.amazonaws.com/h2o/rel-zahradnik/1/Python/h2o-3.30.0.1-py2.py3-none-any.whl

  Using cached https://h2o-release.s3.amazonaws.com/h2o/rel-zahradnik/1/Python/h2o-3.30.0.1-py2.py3-none-any.whl (129.6 MB)


In [ ]:
import h2o
import pandas as pd
h2o.init(port = 54321, ip = "localhost", bind_to_localhost = False, max_mem_size='8G')
h2o.remove_all()

h2odf = h2o.H2OFrame(df,  destination_frame="seven_domain.hex")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,5 hours 47 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,2 years and 17 days !!!
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.680 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
